<a href="https://colab.research.google.com/github/fromakim/2021Election_Analysis/blob/main/data_prepreocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install flatten_json

  Created wheel for flatten-json: filename=flatten_json-0.1.13-py3-none-any.whl size=7980 sha256=eafbff0ea79a377eca1681317f1df852a264f9ef50e53572b557ac9af5b08d0c
  Stored in directory: /root/.cache/pip/wheels/e6/b3/2a/beb2ceb72d11bf335f9c2f87aae26981f6744f3fc885cde665
Successfully built flatten-json


In [1]:
import itertools
import json

In [3]:
import numpy as np
import pandas as pd

In [4]:
from flatten_json import flatten

In [64]:
data = []

In [65]:
for i in range(11):
    f = open(f'./data{i:02}.json', encoding = 'utf-8')
    data.append(json.load(f))

In [66]:
users = pd.concat([pd.DataFrame([flatten(i, root_keys_to_ignore = {'public_metrics', 'entities'}) for i in item['includes']['users']]) for item in data]).reset_index(drop = True)

In [67]:
medias = pd.concat([pd.DataFrame([flatten(i) for i in item['includes']['media']]) if 'media' in item['includes'] else pd.DataFrame() for item in data]).reset_index(drop = True)

In [68]:
tweets = pd.concat([pd.DataFrame([flatten(i, root_keys_to_ignore = {'referenced_tweets', 'public_metrics', 'in_reply_to_user_id', 'attachments', 'entities', 'context_annotations', 'attachments'}) for i in item['includes']['tweets']]) for item in data]).reset_index(drop = True)

In [103]:
user_relations = pd.concat([pd.json_normalize(d['includes']['users']) for d in data]).reset_index(drop = True)[['id', 'entities.url.urls', 'entities.description.urls', 'entities.description.mentions', 'entities.description.hashtags']]

In [91]:
tweet_relations = pd.concat([pd.json_normalize(d['includes']['tweets']) for d in data])[['referenced_tweets', 'entities.urls', 'in_reply_to_user_id']].reset_index(drop = True)

In [104]:
# https://developer.twitter.com/en/docs/twitter-api/users/lookup/api-reference/get-users
user_relations.columns

Index(['id', 'entities.url.urls', 'entities.description.urls',
       'entities.description.mentions', 'entities.description.hashtags'],
      dtype='object')

In [95]:
# https://developer.twitter.com/en/docs/twitter-api/tweets/lookup/api-reference/get-tweets
tweet_relations.columns

Index(['referenced_tweets', 'entities.urls', 'in_reply_to_user_id'], dtype='object')

In [160]:
relation_url_url = user_relations[['id', 'entities.url.urls']].dropna().explode('entities.url.urls')
relation_url_url = pd.concat([relation_url_url, relation_url_url['entities.url.urls'].apply(pd.Series)], axis = 1)
relation_url_url = relation_url_url.drop(columns = 'entities.url.urls').reset_index(drop = True)

In [161]:
relation_url_description = user_relations[['id', 'entities.description.urls']].dropna().explode('entities.description.urls')
relation_url_description = pd.concat([relation_url_description, relation_url_description['entities.description.urls'].apply(pd.Series)], axis = 1)
relation_url_description = relation_url_description.drop(columns = 'entities.description.urls').reset_index(drop = True)

In [165]:
relation_url = pd.concat([relation_url_url, relation_url_description]).reset_index(drop = True)

In [168]:
relation_mention = user_relations[['id', 'entities.description.mentions']].dropna().explode('entities.description.mentions')
relation_mention = pd.concat([relation_mention, relation_mention['entities.description.mentions'].apply(pd.Series)], axis = 1)
relation_mention = relation_mention.drop(columns = 'entities.description.mentions').reset_index(drop = True)

In [178]:
relation_hashtag = user_relations[['id', 'entities.description.hashtags']].dropna().explode('entities.description.hashtags')
relation_hashtag = pd.concat([relation_hashtag, relation_hashtag['entities.description.hashtags'].apply(pd.Series)], axis = 1)
relation_hashtag = relation_hashtag.drop(columns = 'entities.description.hashtags').reset_index(drop = True)